In [1]:
import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import boto3
from boto3.s3.transfer import TransferConfig

In [3]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def LoadFromS3(i):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Load for File",i)
    embeddings = {}
    session = boto3.session.Session(region_name='us-east-1')
    s3client = session.client('s3')
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open(i, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("File Loaded in:", time.time()-now)
    
    return embeddings

In [5]:
def SaveToS3(file, name):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Write for File", name)
    file = file.to_json()
    file.save(f'{name}.h5')
    session = boto3.session.Session(region_name='us-east-1')
    s3 = boto3.resource('s3')
    object = s3.Object('w210-mimic', name)
    object.put(Body=file)

    print("File Saved in:", time.time()-now)
    
    return

In [6]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(8, activation='sigmoid')(LSTM_Layer_1)
opt = keras.optimizers.Adam(lr=0.0001)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
# Saving the Files Locally:

model_json = model.to_json()

with open("adverse_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("adverse_num.h5")


In [18]:
from keras.models import model_from_json

json_file = open('./adverse_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("./adverse_3_embeddings1.hdr")

opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])

print("Loaded model from disk")

Loaded model from disk
Loaded model from disk


In [19]:
EPOCHS = 1

In [20]:
for epoch in range(EPOCHS):
    for i in ['embeddings6', 'embeddings5', 'embeddings4', 'embeddings3', 'embeddings2', 'embeddings1']:
        data = LoadFromS3(i)
        
        data = np.array(list(data.items()))
        HADM_ID_LIST = data[:,0]
        data = data[:,1]

        data = [np.float16(np.concatenate(i[:30])) for i in data]

        embeddings_padded = []
        
        print("Padding Embeddings:")
        for j in data:
            pad_len = 3840 - len(j)
            if pad_len:
                embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
            else:
                embeddings_padded.append(j)
        
        del(data)
        embeddings_padded = np.float16(embeddings_padded)
        embeddings_padded = np.array(embeddings_padded, dtype=np.float16)

        CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/adverse_effect_labels.csv', sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

        EMBEDDINGS_CHAPTER_PIVOT_DF = pd.DataFrame(data=list(zip(HADM_ID_LIST, embeddings_padded)), columns=['HADM_ID', 'embeddings']).set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left')
        EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.fillna(0)
        EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.iloc[:,1:]
        EMBEDDINGS_CHAPTER_PIVOT_NP = np.array(EMBEDDINGS_CHAPTER_PIVOT_DF)
        del(EMBEDDINGS_CHAPTER_PIVOT_DF)

        # Rebalance the Dataset using class weights
#         class_weights = {}
#         class_weight_vals = 1/(EMBEDDINGS_CHAPTER_PIVOT_NP.sum(axis=0)/EMBEDDINGS_CHAPTER_PIVOT_NP.shape[0])

#         for index, weight in enumerate(class_weight_vals):
#             class_weights[index] = int(weight)
        
        print("Training Epoch:", epoch, "Embeddings:",i)
        history = model.fit(embeddings_padded, EMBEDDINGS_CHAPTER_PIVOT_NP, epochs=1, batch_size=256, verbose=1, validation_split=0.1) # ,class_weight=class_weights)
        model.save(f"adverse_{epoch}_{i}.hdr")


Starting Load for File embeddings6
Starting Load for File embeddings6
File Loaded in: 900.7034423351288
File Loaded in: 900.7034423351288
Padding Embeddings:
Padding Embeddings:
Training Epoch: 0 Embeddings: embeddings6
Train on 2453 samples, validate on 273 samples
Epoch 1/1
Training Epoch: 0 Embeddings: embeddings6
Train on 2453 samples, validate on 273 samples
Epoch 1/1
2453/2453 [==============================] - 319s 130ms/step - loss: 0.3180 - get_f1: 0.2744 - val_loss: 0.3590 - val_get_f1: 0.2539
Starting Load for File embeddings5
Starting Load for File embeddings5
File Loaded in: 3490.602745771408
File Loaded in: 3490.602745771408
Padding Embeddings:
Padding Embeddings:
Training Epoch: 0 Embeddings: embeddings5
Train on 9000 samples, validate on 1000 samples
Epoch 1/1
Training Epoch: 0 Embeddings: embeddings5
Train on 9000 samples, validate on 1000 samples
Epoch 1/1
9000/9000 [==============================] - 1264s 140ms/step - loss: 0.3373 - get_f1: 0.3850 - val_loss: 0.3549 

# Cells Below are for Debugging & Legacy